# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_file = "../WeatherPy/Output/cities.csv"
weather_data = pd.read_csv(weather_file)
weather_data.dropna(inplace = True)

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_data[["Lat","Lng"]].astype(float)
Humidity = weather_data["Humidity"].astype(float)

In [11]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=Humidity, dissipating=False, point_radius = 10)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
vacation_cities = weather_data.loc[(weather_data["Max Temp"] <= 78 ) 
                                   & (weather_data["Max Temp"] >= 68)
                                  & (weather_data["Wind Speed"] <= 10)
                                  & (weather_data["Cloudiness"] == 0)]     
vacation_cities

,City,Lat,Lng,Temperature,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
146,kemijarvi,-42.8794,147.3294,71.20,73.00,46.0,0.0,6.91,AU,2021-02-11 00:28:37
198,bengkulu,-17.1078,-43.8150,73.40,73.40,78.0,0.0,5.75,BR,2021-02-11 00:26:36
207,haibowan,10.3042,1.3796,75.20,75.20,23.0,0.0,2.30,BJ,2021-02-11 00:29:54
227,ribeira grande,-41.8697,-73.8203,69.80,69.80,56.0,0.0,4.61,CL,2021-02-11 00:30:19
260,sakaiminato,17.9583,-102.2000,74.93,74.93,76.0,0.0,7.38,MX,2021-02-11 00:31:00
273,kalmar,-24.4066,25.4951,69.80,69.80,100.0,0.0,5.75,BW,2021-02-11 00:31:16
361,port lincoln,-17.3411,-39.2208,76.15,76.15,82.0,0.0,6.73,BR,2021-02-11 00:33:07
370,malazgirt,17.0151,54.0924,73.40,73.40,38.0,0.0,9.22,OM,2021-02-11 00:33:05
407,salalah,16.5167,-98.7500,77.45,77.45,80.0,0.0,8.32,MX,2021-02-11 00:34:00
492,naantali,13.2019,-16.7339,71.60,71.60,56.0,0.0,6.91,GM,2021-02-11 00:35:41


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
hotel_name = []
lat = []
lng = []

In [29]:
# params dictionary to update each iteration
vaca_params = {"radius": 50000,"types": "lodging","key": g_key}

# Use the lat/lng we recovered to identify airports
for index, row in vacation_cities.iterrows():
    
    # get lat, lng from df - this is what isn't working
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    vaca_params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params = vaca_params)

In [ ]:
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_name.append(name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [ ]:
hotel_df = {"Hotel": hotel_name}

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations,info_box_contents=[f"Hotel: {hotel_name}" for 
                                                                 hotel in hotel_df"])
fig.add_layer(markers)
# Display figure
fig